In [1]:
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import time

In [2]:
def cal_distance(x, y):
    #return torch.sum((x-y)**2)**0.5
    #return torch.sum((x-y)**4)**0.5
    #return torch.sum((x-y)**6)**0.5 #best
    #return 1-torch.cosine_similarity(x,y,dim=0) #95.5 k=11
    # return torch.dist(x, y, 6)#best k=6
    return torch.dist(x, y, 2)
    

In [3]:

# iteration func
def KNN_by_iter(train_x, train_y, test_x, test_y, k,dimension):
    pca=PCA(dimension)
    pca.fit(train_x)
    train_x_reduct=torch.from_numpy(pca.transform(train_x))
    test_x_reduct=torch.from_numpy(pca.transform(test_x))
    
    since = time.time()

    # cal distance
    res = []
    for x in tqdm(test_x_reduct):
        dists = []
        for y in train_x_reduct:
            dists.append(cal_distance(x,y).view(1))
        
        idxs = torch.cat(dists).argsort()[:k]
        res.append(np.bincount(np.array([train_y[idx] for idx in idxs])).argmax())

    print(res[:10])#?
    print("acc",accuracy_score(test_y, res))

    time_elapsed = time.time() - since
    print('KNN iter training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        

In [7]:
def Run_Knn(train_dataset,test_x,test_y,dimension):
    train_x = []
    train_y = []
    
    for i in range(len(train_dataset)):
        img, target = train_dataset[i]
        img=img.squeeze()
        
        train_x.append(img.view(-1))
        train_y.append(target)
        if i > 10000:
            break
        
    KNN_by_iter(torch.stack(train_x), train_y, torch.stack(test_x), test_y, 4,dimension)

In [8]:
if __name__ == "__main__":

    train_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=True,download=True)
    test_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=False,download=True)

    test_x = [] 
    test_y = []
    for i in range(len(test_dataset)):
        img, target = test_dataset[i]
        img=img.squeeze()      
        test_x.append(img.view(-1))
        test_y.append(target)

    for dimension in [50,100,200,300,400]:
        Run_Knn(train_dataset,test_x,test_y,dimension)
    
    
    


100%|██████████| 10000/10000 [35:36<00:00,  4.68it/s] 


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9549
KNN iter training complete in 35m 37s


100%|██████████| 10000/10000 [36:01<00:00,  4.63it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9499
KNN iter training complete in 36m 2s


100%|██████████| 10000/10000 [35:09<00:00,  4.74it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9457
KNN iter training complete in 35m 9s


100%|██████████| 10000/10000 [34:44<00:00,  4.80it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9448
KNN iter training complete in 34m 45s


100%|██████████| 10000/10000 [38:05<00:00,  4.38it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9441
KNN iter training complete in 38m 5s
